In [1]:
%%configure -f 
{
    "conf":
    {
        "spark.app.name":"Bhavesh_Demand_Forecast_Notebook",
        "spark.yarn.queue": "workflow",
        "spark.jars": "/apps/Jars/obelisk-retail-legos-Bhavesh.jar,/apps/Jars/mysql-connector-java-5.1.40.jar",
        "spark.driver.extraJavaOptions": "-Dscala.color",
        "spark.driver.memory": "3g",
        "spark.executor.memory": "3g",
        "spark.executor.instances": "4"
    }
}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1415,application_1686980170466_0206,spark,idle,Link,Link,
1416,application_1686980170466_0227,spark,idle,Link,Link,


In [2]:
import ai.couture.obelisk.commons.Constants._
import ai.couture.obelisk.commons.Constants.STANDARD_COL_NAMES._
import ai.couture.obelisk.commons.Constants._
import ai.couture.obelisk.commons.io._
import ai.couture.obelisk.commons.utils.DateTimeUtil._
import ai.couture.obelisk.commons.utils.DataFrameUtil.minMaxScaler
import org.apache.spark.sql._
import org.apache.spark.sql.expressions._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import spark.implicits._

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1417,application_1686980170466_0228,spark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import ai.couture.obelisk.commons.Constants._
import ai.couture.obelisk.commons.Constants.STANDARD_COL_NAMES._
import ai.couture.obelisk.commons.Constants._
import ai.couture.obelisk.commons.io._
import ai.couture.obelisk.commons.utils.DateTimeUtil._
import ai.couture.obelisk.commons.utils.DataFrameUtil.minMaxScaler
import org.apache.spark.sql._
import org.apache.spark.sql.expressions._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import spark.implicits._


In [3]:
var baseDir = "/data/Archive/bhavesh/InventoryPrediction"
var runDate = "2023-04-30"
var runDir = s"$baseDir/$runDate"

// 15 months of train data for arima
var dataStartDate = "2022-02-01"
var predictionStart = "2023-05-01"
var predictionEnd = "2023-05-31"

var trainStart = "2022-04-01"
var trainEnd = "2023-04-29"

var trainXStart = "2022-04-01"
var trainXEnd = "2023-03-30"
var trainYStart = "2023-04-01"
var trainYEnd = "2023-04-29"

var testXStart = "2022-05-01"
var testXEnd = "2023-04-29"
var testYStart = "2023-05-01"
var testYEnd = "2023-05-31"

var suffix = "colorfamily_pricebucket_styletype_pattern_sleeve_brandname"

val PREDICTED="predicted"
val ACTUAL="actual"

val getOldDateFromHereUDF = udf((numDays: Int, date: String, inputDateFormat: String, outputDateFormat: String) => {
    getOldDateFromHere(numDays, date, inputDateFormat, outputDateFormat)
})

def getMonths(dateFormat: String)(df: DataFrame): DataFrame = {
    df.withColumn("dateFormatted", getOldDateFromHereUDF(lit(0), col(DATE), lit(dateFormat), lit("yyyy-MM-dd")))
    .withColumn("year", split(col("dateFormatted"), "-")(0))
    .withColumn("month", split(col("dateFormatted"), "-")(1))
    .withColumn("monthsInHis", dense_rank().over(Window.orderBy(desc("year"), desc("month"))) - lit(1))
    .drop("dateFormatted", "year", "month")
}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

baseDir: String = /data/Archive/bhavesh/InventoryPrediction
runDate: String = 2023-04-30
runDir: String = /data/Archive/bhavesh/InventoryPrediction/2023-04-30
dataStartDate: String = 2022-02-01
predictionStart: String = 2023-05-01
predictionEnd: String = 2023-05-31
trainStart: String = 2022-04-01
trainEnd: String = 2023-04-29
trainXStart: String = 2022-04-01
trainXEnd: String = 2023-03-30
trainYStart: String = 2023-04-01
trainYEnd: String = 2023-04-30
testXStart: String = 2022-05-01
testXEnd: String = 2023-04-29
testYStart: String = 2023-05-01
testYEnd: String = 2023-05-31
suffix: String = colorfamily_pricebucket_styletype_pattern_sleeve_brandname
PREDICTED: String = predicted
ACTUAL: String = actual
getOldDateFromHereUDF: org.apache.spark.sql.expressions.UserDefinedFunction = UserDefinedFunction(<function4>,StringType,Some(List(IntegerType, StringType, StringType, StringType)))
getMonths: (dateFormat: String)(df: org.apache.spark.sql.DataFrame)org.apache.spark.sql.DataFrame


### Product Attributes && QueryProductMap

In [4]:
var productAttrs = ParquetToDF.getDF("/data/Archive/inventory/2023-04-30/queryProductMap_colorfamily_pricebucket_styletype_pattern_sleeve_brandname/processedProductAttr")
productAttrs.printSchema

DFToParquet.putDF(s"$runDir/ProductAttributes", productAttrs)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
var queryToProductMap = ParquetToDF.getDF(s"/data/ecomm/ajio/processed/inventory/2023-04-30/queryProductMap_$suffix")
queryToProductMap.printSchema

DFToParquet.putDF(s"$runDir/QueryProductMapping/suffix=$suffix", queryToProductMap)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Product Level Daily/Monthly Quantity

In [4]:
var productAttrs = ParquetToDF.getDF(s"$runDir/ProductAttributes").select(PRODUCTID).distinct()

var interactionsDB = (
    ParquetToDF.getDF("/data/ecomm/ajio/processed/interactionsDB")
    .filter(col("event") === "Checkout" && col("purchase") === "New" 
            && col("productid").isNotNull && col("booked_rev") > 0 
            && col("userid").isNotNull && col("quantity") > 0 
            && col("date")<=predictionEnd && col("date")>=dataStartDate
            && col("date") =!= runDate
           )
    .groupBy("productid", "date")
    .agg(sum("quantity").cast(DoubleType).as("quantity"))
)

DFToParquet.putDF(s"$runDir/DailyQuantity/product", interactionsDB.join(productAttrs, Seq(PRODUCTID)))

var productDailyQuantity = ParquetToDF.getDF(s"$runDir/DailyQuantity/product")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

productAttrs: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [productid: string]
interactionsDB: org.apache.spark.sql.DataFrame = [productid: string, date: date ... 1 more field]
productDailyQuantity: org.apache.spark.sql.DataFrame = [productid: string, date: date ... 1 more field]


In [5]:
var monthMap = ParquetToDF.getDF("/data/Archive/bhavesh/InventoryPrediction/2023-04-30/MonthsInHistoryMap").select(col("monthsInHis"), col("monthStartDate").as("tempDateForTimeSeries"))

var productMonthlyQuantity = (
    ParquetToDF.getDF(s"$runDir/DailyQuantity/product").transform(getMonths("yyyy-MM-dd")).filter(col("monthsInHis") <= 15)
    .groupBy(PRODUCTID, "monthsInHis")
    .agg(sum("quantity") as "quantity")
    .join(monthMap, Seq("monthsInHis"))
)

DFToParquet.putDF(s"$runDir/MonthlyQuantity/product", productMonthlyQuantity.repartition(210))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

monthMap: org.apache.spark.sql.DataFrame = [monthsInHis: int, tempDateForTimeSeries: date]
productMonthlyQuantity: org.apache.spark.sql.DataFrame = [monthsInHis: int, productid: string ... 2 more fields]


In [6]:
var queryProductMap = ParquetToDF.getDF(s"$runDir/QueryProductMapping/suffix=$suffix").na.drop()

var queryDailyQuantity = (
    ParquetToDF.getDF(s"$runDir/DailyQuantity/product")
    .join(queryProductMap, Seq(PRODUCTID))
    .groupBy("similargrouplevel", "query", "template", "date")
    .agg(sum("quantity") as "quantity")
)
DFToParquet.putDF(s"$runDir/DailyQuantity/query", queryDailyQuantity)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

queryProductMap: org.apache.spark.sql.DataFrame = [query: string, productid: string ... 2 more fields]
queryDailyQuantity: org.apache.spark.sql.DataFrame = [similargrouplevel: int, query: string ... 3 more fields]


In [ ]:
var monthMap = ParquetToDF.getDF("/data/Archive/bhavesh/InventoryPrediction/2023-04-30/MonthsInHistoryMap").select(col("monthsInHis"), col("monthStartDate").as("tempDateForTimeSeries"))

var queryMonthlyQuantity = (
    ParquetToDF.getDF(s"$runDir/DailyQuantity/query").transform(getMonths("yyyy-MM-dd")).filter(col("monthsInHis") <= 15)
    .groupBy("similargrouplevel", "query", "template", "monthsInHis")
    .agg(sum("quantity") as "quantity")
    .join(monthMap, Seq("monthsInHis"))
)
DFToParquet.putDF(s"$runDir/MonthlyQuantity/query", queryMonthlyQuantity.repartition(210))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Prediction space Products/queries

In [ ]:
var productMonthlyQuantity = ParquetToDF.getDF(s"$runDir/MonthlyQuantity/product")

var productsToPredict = productMonthlyQuantity.filter(col("monthsInHis") >= 1 && col("monthsInHis") <= 12).select(PRODUCTID).distinct()
DFToParquet.putDF(s"$runDir/PredictionSpace/products", productsToPredict)

In [12]:
var queryProductMap = ParquetToDF.getDF(s"$runDir/QueryProductMapping/suffix=$suffix").na.drop().filter(col("query") =!= "")

var queriesToPredict = (
    ParquetToDF.getDF(s"$runDir/PredictionSpace/products")
    .join(queryProductMap, Seq(PRODUCTID))
    .select("similargrouplevel", "query", "template")
    .distinct()
)
DFToParquet.putDF(s"$runDir/PredictionSpace/queries", queriesToPredict)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

queryProductMap: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [query: string, productid: string ... 2 more fields]
queriesToPredict: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [similargrouplevel: int, query: string ... 1 more field]


### Standard deviation for products/queries based on 12 months data

In [ ]:
var productMonthlyQuantity = ParquetToDF.getDF(s"$runDir/MonthlyQuantity/product")

var productSigma = (
    productMonthlyQuantity.filter(col("monthsInHis") >= 1 && col("monthsInHis") <=12)
    .drop("tempDateForTimeSeries")
    .groupBy(PRODUCTID).agg(stddev("quantity").as("stddev"),avg("quantity").as("avg")).na.fill(0.0, Seq("stddev"))
)

DFToParquet.putDF(s"$runDir/Statistics12Months/product", productSigma)

In [ ]:
var queryMonthlyQuantity = ParquetToDF.getDF(s"$runDir/MonthlyQuantity/query")

var querySigma = (
    queryMonthlyQuantity.filter(col("monthsInHis") >= 1 && col("monthsInHis") <=12)
    .drop("tempDateForTimeSeries")
    .groupBy("similargrouplevel", "query", "template").agg(stddev("quantity").as("stddev"),avg("quantity").as("avg")).na.fill(0.0, Seq("stddev"))
)

DFToParquet.putDF(s"$runDir/Statistics12Months/query", querySigma)

### Actual Sales

In [ ]:
var actualProduct = (
    ParquetToDF.getDF(s"$runDir/MonthlyQuantity/product")
    .filter(col("monthsInHis") === 0)
    .select(col(PRODUCTID), col("quantity").alias("sales"))
)

DFToParquet.putDF(s"$runDir/Actual/sales/product", actualProduct)

var queryProductMap = ParquetToDF.getDF(s"$runDir/QueryProductMapping/suffix=$suffix").na.drop()
var actualQuery = (
    ParquetToDF.getDF(s"$runDir/Actual/sales/product")
    .join(queryProductMap, Seq(PRODUCTID))
    .groupBy("similargrouplevel", "query", "template")
    .agg(sum("sales") as "sales")
)

DFToParquet.putDF(s"$runDir/Actual/sales/query", actualQuery)

In [13]:
var actualQuery = ParquetToDF.getDF(s"$runDir/Actual/sales/query")
actualQuery.printSchema

var queriesToPredict = ParquetToDF.getDF(s"$runDir/PredictionSpace/queries")
queriesToPredict.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

actualQuery: org.apache.spark.sql.DataFrame = [similargrouplevel: int, query: string ... 2 more fields]
root
 |-- similargrouplevel: integer (nullable = true)
 |-- query: string (nullable = true)
 |-- template: string (nullable = true)
 |-- sales: double (nullable = true)

queriesToPredict: org.apache.spark.sql.DataFrame = [similargrouplevel: int, query: string ... 1 more field]
root
 |-- similargrouplevel: integer (nullable = true)
 |-- query: string (nullable = true)
 |-- template: string (nullable = true)



### Generate target

In [24]:
var target = ParquetToDF.getDF(s"$runDir/MonthlyQuantity/product").filter(col("monthsInHis") === 1).select(col("productid"), col("quantity").alias("yQuantity"))
target.printSchema

DFToParquet.putDF(s"$runDir/dataset/train/target", target)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

target: org.apache.spark.sql.DataFrame = [productid: string, yQuantity: double]
root
 |-- productid: string (nullable = true)
 |-- yQuantity: double (nullable = true)



### Generate Embeddings

In [53]:
var df = ParquetToDF.getDF("/data/Archive/bhavesh/inventoryPrediction-deprecated/embeddings/data")
df.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

df: org.apache.spark.sql.DataFrame = [productid: string, yQuantity: bigint ... 6 more fields]
root
 |-- productid: string (nullable = true)
 |-- yQuantity: long (nullable = true)
 |-- similargrouplevel: string (nullable = true)
 |-- colorfamily: string (nullable = true)
 |-- brandname: string (nullable = true)
 |-- sleeve: string (nullable = true)
 |-- pattern: string (nullable = true)
 |-- styletype: string (nullable = true)



In [6]:
var target = ParquetToDF.getDF(s"$runDir/dataset/train/target")
target.printSchema

var productAttrs = ParquetToDF.getDF(s"$runDir/ProductAttributes").select(PRODUCTID, "similargrouplevel", "colorfamily", "brandname", "sleeve", "pattern", "styletype")
productAttrs.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

target: org.apache.spark.sql.DataFrame = [productid: string, yQuantity: double]
root
 |-- productid: string (nullable = true)
 |-- yQuantity: double (nullable = true)

productAttrs: org.apache.spark.sql.DataFrame = [productid: string, similargrouplevel: string ... 5 more fields]
root
 |-- productid: string (nullable = true)
 |-- similargrouplevel: string (nullable = true)
 |-- colorfamily: string (nullable = true)
 |-- brandname: string (nullable = true)
 |-- sleeve: string (nullable = true)
 |-- pattern: string (nullable = true)
 |-- styletype: string (nullable = true)



In [8]:
var embeddingsData = productAttrs.join(target, Seq(PRODUCTID))
embeddingsData.printSchema

DFToParquet.putDF(s"$runDir/Embeddings/data", embeddingsData)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

embeddingsData: org.apache.spark.sql.DataFrame = [productid: string, similargrouplevel: string ... 6 more fields]
root
 |-- productid: string (nullable = true)
 |-- similargrouplevel: string (nullable = true)
 |-- colorfamily: string (nullable = true)
 |-- brandname: string (nullable = true)
 |-- sleeve: string (nullable = true)
 |-- pattern: string (nullable = true)
 |-- styletype: string (nullable = true)
 |-- yQuantity: double (nullable = true)



### Dataset Features

In [4]:
var datasetDir = s"$runDir/dataset"
var productAttrs = (
    ParquetToDF.getDF(s"$runDir/ProductAttributes").select(col(PRODUCTID), col(SIMILAR_GROUP_LEVEL))
    .dropDuplicates(PRODUCTID)
    .filter(col(SIMILAR_GROUP_LEVEL).isin("830216013", "830303011"))
    .persist()
)

def saveNLoadDF(path: String)(df: DataFrame): DataFrame = {
    DFToParquet.putDF(path, df)
    ParquetToDF.getDF(path)
}

def getMonthlyMeanStd(monthsLimit: Int, colsForStatistics: Array[String], calculateStd: Boolean)(df: DataFrame): DataFrame = {
    var intermediateDF = df
    val monthsArray = (0 until monthsLimit).toArray
    val colsToUseForCalculations = colsForStatistics.map(column => (column, monthsArray.map(month => s"${month}_${column}"))).toMap
    colsToUseForCalculations.foreach(
        columnInfo => {
            val attr = columnInfo._1
            val colsToUse = columnInfo._2
            val meanExpr = colsToUse.map(col).reduce((col1, col2) => col1 + col2)
            intermediateDF = intermediateDF.withColumn(s"${attr}_avg_${monthsLimit}_months", meanExpr/lit(colsToUse.length).cast(DoubleType))
            
            if(calculateStd){
                print(attr)
                val stdExpr = colsToUse.map(column => pow(col(column) - col(s"${attr}_avg_${monthsLimit}_months"), lit(2)))
                .reduce((col1, col2) => col1 + col2)
                intermediateDF = intermediateDF.withColumn(s"${attr}_std_${monthsLimit}_months",
                                                           sqrt(stdExpr/lit(colsToUse.length - 1)))
            }
        }
    )
    intermediateDF
}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

datasetDir: String = /data/Archive/bhavesh/InventoryPrediction/2023-04-30/dataset
productAttrs: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [productid: string, similargrouplevel: string]
saveNLoadDF: (path: String)(df: org.apache.spark.sql.DataFrame)org.apache.spark.sql.DataFrame
getMonthlyMeanStd: (monthsLimit: Int, colsForStatistics: Array[String], calculateStd: Boolean)(df: org.apache.spark.sql.DataFrame)org.apache.spark.sql.DataFrame


In [17]:
def generateSalesXData(of: String, startDate: String, endDate: String, productAttrs: DataFrame, baseDir: String): Unit = {

    val features = Array("sales")
    val tempDir = "/data/Archive/bhavesh/InventoryPrediction/temp/generateSalesXData"
    
    var interactions = ParquetToDF.getDF("/data/ecomm/ajio/processed/interactionsDB")
    .filter(col("event") === "Checkout" && col("purchase") === "New" 
            && col("productid").isNotNull && col("booked_rev") > 0 
            && col("userid").isNotNull && col("quantity") > 0
            && col(DATE).between(startDate, endDate)
           )
    .groupBy(PRODUCTID, DATE)
    .agg(sum(QUANTITY) as "sales")
    .join(productAttrs, Seq(PRODUCTID))
    .transform(saveNLoadDF(s"$tempDir/interactions_1"))
    .transform(getMonths("yyyy-MM-dd"))
    .filter(col("monthsInHis") < 12)
    .groupBy(PRODUCTID)
    .pivot("monthsInHis")
    .agg(sum("sales").cast(DoubleType) as "sales", first("sales") as "temp")
    
    val colsToDrop = interactions.columns.filter(_.endsWith("temp"))
    interactions = interactions.drop(colsToDrop: _*).na.fill(0)
    .transform(saveNLoadDF(s"$tempDir/interactions_2"))
    .transform(getMonthlyMeanStd(3, Array("sales"), false))
    .transform(getMonthlyMeanStd(6, Array("sales"), false))
    .transform(getMonthlyMeanStd(12, Array("sales"), true))
    
    DFToParquet.putDF(s"$baseDir/$of/sales", interactions)
    HdfsUtils.delete(tempDir)
}

generateSalesXData("train", trainXStart, trainXEnd, productAttrs, datasetDir)
generateSalesXData("test", testXStart, testXEnd, productAttrs, datasetDir)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

generateSalesXData: (of: String, startDate: String, endDate: String, productAttrs: org.apache.spark.sql.DataFrame, baseDir: String)Unit


In [18]:
def generateGAXData(of: String, startDate: String, endDate: String, productAttrs: DataFrame, baseDir: String): Unit = {

    val features = Array("PLPViewsPerDay", "PLPClicksPerDay", "PDPCountPerDay", "TotalAddToCartPerDay")
    val aggExprs = features.map(feature => sum(feature).cast(DoubleType).alias(feature))
    val tempDir = "/data/Archive/bhavesh/InventoryPrediction/temp/generateGAXData"
    
    val ga = ParquetToDF.getDF("/data/ecomm/ajio/processed/ProcessedGAData")
    .filter(col(DATE).between(startDate, endDate))
    .groupBy(PRODUCTID, DATE)
    .agg(aggExprs.head, aggExprs.tail: _*)
    .join(productAttrs, Seq(PRODUCTID))
    .transform(saveNLoadDF(s"$tempDir/ga_1"))
    .transform(getMonths("yyyy-MM-dd"))
    .filter(col("monthsInHis") < 12)
    .groupBy(PRODUCTID)
    .pivot("monthsInHis")
    .agg(aggExprs.head, aggExprs.tail: _*)
    .na.fill(0)
    .transform(saveNLoadDF(s"$tempDir/ga_2"))
    .transform(getMonthlyMeanStd(3, features, false))
    .transform(getMonthlyMeanStd(6, features, false))
    .transform(getMonthlyMeanStd(12, features, true))
    
    DFToParquet.putDF(s"$baseDir/$of/ga", ga)
    HdfsUtils.delete(tempDir)
}

generateGAXData("train", trainXStart, trainXEnd, productAttrs, datasetDir)
generateGAXData("test", testXStart, testXEnd, productAttrs, datasetDir)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

generateGAXData: (of: String, startDate: String, endDate: String, productAttrs: org.apache.spark.sql.DataFrame, baseDir: String)Unit


In [19]:
def generateWishlistData(of: String, startDate: String, endDate: String, productAttrs: DataFrame, baseDir: String): Unit = {
    val features = Array("wishlist")
    val tempDir = "/data/Archive/bhavesh/InventoryPrediction/temp/generateWishlistData"
    
    var wishlist = ParquetToDF.getDF("/data/ecomm/ajio/processed/processedWishlist")
    .filter(col(DATE).between(startDate, endDate))
    .groupBy(PRODUCTID, DATE)
    .agg(countDistinct("wishlistid").cast(DoubleType) as "wishlist")
    .join(productAttrs, Seq(PRODUCTID))
    .transform(saveNLoadDF(s"$tempDir/wishlist_1"))
    .transform(getMonths("yyyy-MM-dd"))
    .filter(col("monthsInHis") < 12)
    .groupBy(PRODUCTID)
    .pivot("monthsInHis")
    .agg(sum("wishlist") as "wishlist", first("wishlist") as "temp")
    
    val colsToDrop = wishlist.columns.filter(_.endsWith("temp"))
    wishlist = wishlist.drop(colsToDrop: _*).na.fill(0)
    .transform(saveNLoadDF(s"$tempDir/wishlist_2"))
    .transform(getMonthlyMeanStd(3, features, false))
    .transform(getMonthlyMeanStd(6, features, false))
    // .transform(getMonthlyMeanStd(12, features, true))
    
    DFToParquet.putDF(s"$baseDir/$of/wishlist", wishlist)
    HdfsUtils.delete(tempDir)
}
generateWishlistData("train", trainXStart, trainXEnd, productAttrs, datasetDir)
generateWishlistData("test", testXStart, testXEnd, productAttrs, datasetDir)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

generateWishlistData: (of: String, startDate: String, endDate: String, productAttrs: org.apache.spark.sql.DataFrame, baseDir: String)Unit


In [20]:
def generateReturnsData(of: String, startDate: String, endDate: String, productAttrs: DataFrame, baseDir: String): Unit = {
    val features = Array("returns")
    val tempDir = "/data/Archive/bhavesh/InventoryPrediction/temp/generateReturns"
    
    var returns = ParquetToDF.getDF("/data/ecomm/ajio/processed/interactionsDB")
    .filter(col("event") === "Return" && col(QUANTITY) >= 0 &&
        col(USERID).isNotNull && col(PRODUCTID).isNotNull)
    .filter(col(DATE).between(startDate, endDate))
    .groupBy(PRODUCTID, DATE)
    .agg(sum(QUANTITY).cast(DoubleType) as "returns")
    .join(productAttrs, Seq(PRODUCTID))
    .transform(saveNLoadDF(s"$tempDir/returns_1"))
    .transform(getMonths("yyyy-MM-dd"))
    .filter(col("monthsInHis") < 12)
    .groupBy(PRODUCTID)
    .pivot("monthsInHis")
    .agg(sum("returns") as "returns", first("returns") as "temp")
    
    val colsToDrop = returns.columns.filter(_.endsWith("temp"))
    returns = returns.drop(colsToDrop: _*).na.fill(0)
    .transform(saveNLoadDF(s"$tempDir/returns_2"))
    .transform(getMonthlyMeanStd(3, features, false))
    .transform(getMonthlyMeanStd(6, features, false))
    .transform(getMonthlyMeanStd(12, features, true))
    
    DFToParquet.putDF(s"$baseDir/$of/returns", returns)
    HdfsUtils.delete(tempDir)
}
generateReturnsData("train", trainXStart, trainXEnd, productAttrs, datasetDir)
generateReturnsData("test", testXStart, testXEnd, productAttrs, datasetDir)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

generateReturnsData: (of: String, startDate: String, endDate: String, productAttrs: org.apache.spark.sql.DataFrame, baseDir: String)Unit


In [21]:
def generateUsersData(of: String, startDate: String, endDate: String, productAttrs: DataFrame, baseDir: String): Unit = {
    val features = Array("users")
    val tempDir = "/data/Archive/bhavesh/InventoryPrediction/temp/generateUsersData"
    
    var interactions = ParquetToDF.getDF("/data/ecomm/ajio/processed/interactionsDB")
    .filter(col("event") === "Checkout" && col("purchase") === "New" 
            && col("productid").isNotNull && col("booked_rev") > 0 
            && col("userid").isNotNull && col("quantity") > 0
            && col(DATE).between(startDate, endDate)
           )
    .join(productAttrs, Seq(PRODUCTID))
    .transform(saveNLoadDF(s"$tempDir/interactions_1"))
    .transform(getMonths("yyyy-MM-dd"))
    .filter(col("monthsInHis") < 12)
    .groupBy(PRODUCTID)
    .pivot("monthsInHis")
    .agg(countDistinct(USERID).cast(DoubleType) as "users", first(USERID) as "temp")
    
    val colsToDrop = interactions.columns.filter(_.endsWith("temp"))
    interactions = interactions.drop(colsToDrop: _*).na.fill(0)
    .transform(saveNLoadDF(s"$tempDir/interactions_2"))
    .transform(getMonthlyMeanStd(3, features, false))
    .transform(getMonthlyMeanStd(6, features, false))
    .transform(getMonthlyMeanStd(12, features, true))
    
    DFToParquet.putDF(s"$baseDir/$of/users", interactions)
    HdfsUtils.delete(tempDir)
}
generateUsersData("train", trainXStart, trainXEnd, productAttrs, datasetDir)
generateUsersData("test", testXStart, testXEnd, productAttrs, datasetDir)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

generateUsersData: (of: String, startDate: String, endDate: String, productAttrs: org.apache.spark.sql.DataFrame, baseDir: String)Unit


In [22]:
def generateAvailabilityQuantityData(of: String, startDate: String, endDate: String, productAttrs: DataFrame, baseDir: String): Unit = {
    val features = Array("totalAvailableQuantity", "maxAvailableQuantity")
    val tempDir = "/data/Archive/bhavesh/InventoryPrediction/temp/generateAvailabilityQuantityData"
    
    var inventory = ParquetToDF.getDF("/data/ecomm/ajio/processed/LiveProductsInventoryHistoryLegos")
    .withColumnRenamed(ITEM_ID, PRODUCTID)
    .filter(col(DATE).between(startDate, endDate))
    .groupBy(PRODUCTID, DATE)
    .agg(sum("availablequantity").as("availableQuantity"))
    .join(productAttrs, Seq(PRODUCTID))
    .transform(saveNLoadDF(s"$tempDir/inventory_1"))
    .transform(getMonths("yyyy-MM-dd"))
    .filter(col("monthsInHis") < 12)
    .groupBy(PRODUCTID)
    .pivot("monthsInHis")
    .agg(sum("availableQuantity").cast(DoubleType) as "totalAvailableQuantity", max("availableQuantity") as "maxAvailableQuantity")
    
    inventory = inventory.na.fill(0)
    .transform(saveNLoadDF(s"$tempDir/inventory_2"))
    .transform(getMonthlyMeanStd(3, features, false))
    .transform(getMonthlyMeanStd(6, features, false))
    .transform(getMonthlyMeanStd(12, features, true))
    
    DFToParquet.putDF(s"$baseDir/$of/availableQuantity", inventory)
    HdfsUtils.delete(tempDir)
}
generateAvailabilityQuantityData("train", trainXStart, trainXEnd, productAttrs, datasetDir)
generateAvailabilityQuantityData("test", testXStart, testXEnd, productAttrs, datasetDir)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

generateAvailabilityQuantityData: (of: String, startDate: String, endDate: String, productAttrs: org.apache.spark.sql.DataFrame, baseDir: String)Unit


### Dataset

In [30]:
def CreateDataset(of: String, xPath: String, yPath: String, basePath: String): Unit = {
    val tempDir = "/data/Archive/bhavesh/InventoryPrediction/temp/createDataset"
    var xData = ParquetToDF.getDF(s"$xPath/$of/sales").drop(SIMILAR_GROUP_LEVEL)
    .join(ParquetToDF.getDF(s"$xPath/$of/ga").drop(SIMILAR_GROUP_LEVEL), Seq(PRODUCTID), "outer")
    .transform(saveNLoadDF(s"$tempDir/dataset_1"))
    .join(ParquetToDF.getDF(s"$xPath/$of/wishlist").drop(SIMILAR_GROUP_LEVEL), Seq(PRODUCTID), "outer")
    .join(ParquetToDF.getDF(s"$xPath/$of/returns").drop(SIMILAR_GROUP_LEVEL), Seq(PRODUCTID), "outer")
    .transform(saveNLoadDF(s"$tempDir/dataset_2"))
    .join(ParquetToDF.getDF(s"$xPath/$of/users").drop(SIMILAR_GROUP_LEVEL), Seq(PRODUCTID), "outer")
    .join(ParquetToDF.getDF(s"$xPath/$of/availableQuantity").drop(SIMILAR_GROUP_LEVEL), Seq(PRODUCTID), "outer")
    .na.fill(0)
    
    if(of == "test"){
        var productsToPredict = ParquetToDF.getDF("/data/Archive/bhavesh/InventoryPrediction/2023-04-30/PredictionSpace/products")
        xData = productsToPredict.join(xData, Seq(PRODUCTID), "left").na.fill(0)
    }
    
    DFToParquet.putDF(s"$basePath/$of/XData", xData)
    
    if(of == "train"){
        var yData = ParquetToDF.getDF(s"$yPath/$of/target").drop(SIMILAR_GROUP_LEVEL)
        DFToParquet.putDF(s"$basePath/$of/YData", yData)
    }
    HdfsUtils.delete(tempDir)
}
CreateDataset("train", datasetDir, datasetDir, s"$runDir/OriginalDataset")
CreateDataset("test", datasetDir, datasetDir, s"$runDir/OriginalDataset")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

CreateDataset: (of: String, xPath: String, yPath: String, basePath: String)Unit


### Transform dataset

In [27]:
def generateTransformedDataset(of: String, datasetPath: String, transformedDatasetPath: String, performNormalization: Boolean = true, isBaselineTransformation: Boolean = true): Unit = {
    var xData: DataFrame = ParquetToDF.getDF(s"$datasetPath/$of/XData")
    
    if(isBaselineTransformation){
        xData = (
            xData
            .select("productid", "0_sales", "1_sales", "2_sales", "3_sales", "4_sales", "5_sales", "6_sales", "7_sales", "8_sales", "9_sales", "10_sales", "11_sales",
               "sales_avg_12_months", "sales_std_12_months", "0_returns", "0_users", "0_PLPViewsPerDay", "0_PLPClicksPerDay", "0_PDPCountPerDay",
               "0_TotalAddToCartPerDay", "0_wishlist", "0_totalAvailableQuantity")
            .toDF("productid", "0_monthSales", "1_monthSales", "2_monthSales", "3_monthSales", "4_monthSales", "5_monthSales", "6_monthSales", "7_monthSales", "8_monthSales", 
              "9_monthSales", "10_monthSales", "11_monthSales", "avgSales", "stddevSales", "totalReturn", "totalUsers", "totalPLPViews", "totalPLPClicks", 
              "totalPDPCount", "totalATC", "totalWishList", "totalAvailableQuantity")
        )
    }
    
    
    var data: DataFrame = null
    if (of == "train") {
        var yData: DataFrame = ParquetToDF.getDF(s"$datasetPath/$of/YData")
        data = xData.join(yData, Seq(PRODUCTID))
    }
    else if(of == "test"){
        data = xData
    }
    
    val productAttributesLegosFNL: DataFrame = (
        ParquetToDF.getDF(s"$runDir/ProductAttributes")
        .select(PRODUCTID, SIMILAR_GROUP_LEVEL, "colorfamily", "brandname", "styletype", "pattern", "sleeve", "pricebucket")
    )
    
    data = data.join(productAttributesLegosFNL, Seq(PRODUCTID)).na.fill("Null")
    
    println(data.count)
    var attributes = Array("colorfamily", "pattern", "brandname", "styletype", "sleeve")
    attributes.foreach(
        attribute => {
            var attrDFMS = CSVToDF.getDF(s"$runDir/Embeddings/menShirts/${attribute}.csv", inferSchema=true)
            
            attrDFMS = attrDFMS
            .select(attribute, attrDFMS.columns.filter(_.startsWith("normalized")): _*)
            .withColumn(SIMILAR_GROUP_LEVEL, lit("830216013"))
            
            var attrDFWK = CSVToDF.getDF(s"$runDir/Embeddings/womenKurtas/${attribute}.csv", inferSchema=true)
            
            attrDFWK = attrDFWK
            .select(attribute, attrDFMS.columns.filter(_.startsWith("normalized")): _*)
            .withColumn(SIMILAR_GROUP_LEVEL, lit("830303011"))
            
            var cols = attrDFWK.columns
            
            var attrDF = attrDFMS.select(cols.head, cols.tail: _*).union(attrDFWK)
            
            data = data.join(attrDF, Seq(SIMILAR_GROUP_LEVEL, attribute)).drop(attribute)
        }
    )
    
    println(data.count)
    val menShirtsData: DataFrame = data.filter(col(SIMILAR_GROUP_LEVEL) === "830216013")
    val womenKurtasData: DataFrame = data.filter(col(SIMILAR_GROUP_LEVEL) === "830303011")
    
    DFToParquet.putDF(s"$transformedDatasetPath/combined/data/$of", data)
    DFToParquet.putDF(s"$transformedDatasetPath/menShirts/data/$of", menShirtsData)
    DFToParquet.putDF(s"$transformedDatasetPath/womenKurtas/data/$of", womenKurtasData)
    
    if (performNormalization){
        
        var colsToScale: Array[String] = Array(
            "0_monthSales", "1_monthSales", "2_monthSales", "3_monthSales", "4_monthSales", "5_monthSales", "6_monthSales", "7_monthSales", 
            "8_monthSales", "9_monthSales", "10_monthSales", "11_monthSales", "avgSales", "stddevSales", "totalReturn", "totalUsers", 
            "totalPLPViews", "totalPLPClicks", "totalPDPCount", "totalATC", "totalWishList", "totalAvailableQuantity", "pricebucket")

        if(of == "train"){

            colsToScale = colsToScale ++ Array("yQuantity")
        }

        DFToParquet.putDF(s"$transformedDatasetPath/combined/normalizedData/$of", data.transform(minMaxScaler(colsToScale)))
        DFToParquet.putDF(s"$transformedDatasetPath/menShirts/normalizedData/$of", menShirtsData.transform(minMaxScaler(colsToScale)))
        DFToParquet.putDF(s"$transformedDatasetPath/womenKurtas/normalizedData/$of", womenKurtasData.transform(minMaxScaler(colsToScale)))
    }
}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

generateTransformedDataset: (of: String, datasetPath: String, transformedDatasetPath: String, performNormalization: Boolean, isBaselineTransformation: Boolean)Unit


In [14]:
generateTransformedDataset("train", s"$runDir/OriginalDataset", s"$runDir/TransformedDataset/baseline", true, true)
generateTransformedDataset("test", s"$runDir/OriginalDataset", s"$runDir/TransformedDataset/baseline", true, true)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
generateTransformedDataset("train", s"$runDir/OriginalDataset", s"$runDir/TransformedDataset/complete", false, false)
generateTransformedDataset("test", s"$runDir/OriginalDataset", s"$runDir/TransformedDataset/complete", false, false)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Dataset Analysis

In [25]:
productsToPredict.count

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res31: Long = 157640


In [30]:
productAttrs = productAttrs.select("productid", "similargrouplevel", "colorfamily", "brandname", "sleeve", "pattern", "styletype")
productAttrs.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

productAttrs: org.apache.spark.sql.DataFrame = [productid: string, similargrouplevel: string ... 5 more fields]
root
 |-- productid: string (nullable = true)
 |-- similargrouplevel: string (nullable = true)
 |-- colorfamily: string (nullable = true)
 |-- brandname: string (nullable = true)
 |-- sleeve: string (nullable = true)
 |-- pattern: string (nullable = true)
 |-- styletype: string (nullable = true)



In [31]:
productsToPredict = productsToPredict.join(productAttrs, Seq(PRODUCTID)).na.fill("Null")
productsToPredict.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

productsToPredict: org.apache.spark.sql.DataFrame = [productid: string, similargrouplevel: string ... 5 more fields]
root
 |-- productid: string (nullable = false)
 |-- similargrouplevel: string (nullable = false)
 |-- colorfamily: string (nullable = false)
 |-- brandname: string (nullable = false)
 |-- sleeve: string (nullable = false)
 |-- pattern: string (nullable = false)
 |-- styletype: string (nullable = false)



In [32]:
productsToPredict.count

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res37: Long = 157640


In [39]:
var attribute = "sleeve"
var attrDFMS = CSVToDF.getDF(s"$runDir/Embeddings/menShirts/${attribute}.csv", inferSchema=true)
            
attrDFMS = (attrDFMS
.select(attribute, attrDFMS.columns.filter(_.startsWith("normalized")): _*)
.withColumn(SIMILAR_GROUP_LEVEL, lit("830216013")))

var attrDFWK = CSVToDF.getDF(s"$runDir/Embeddings/womenKurtas/${attribute}.csv", inferSchema=true)

attrDFWK = (attrDFWK
.select(attribute, attrDFMS.columns.filter(_.startsWith("normalized")): _*)
.withColumn(SIMILAR_GROUP_LEVEL, lit("830303011")))

var cols = attrDFWK.columns

var attrDF = attrDFMS.select(cols.head, cols.tail: _*).union(attrDFWK)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

attribute: String = sleeve
attrDFMS: org.apache.spark.sql.DataFrame = [sleeve_0: double, sleeve_1: double ... 4 more fields]
attrDFMS: org.apache.spark.sql.DataFrame = [sleeve: string, normalized_sleeve_0: double ... 2 more fields]
attrDFWK: org.apache.spark.sql.DataFrame = [sleeve_0: double, sleeve_1: double ... 4 more fields]
attrDFWK: org.apache.spark.sql.DataFrame = [sleeve: string, normalized_sleeve_0: double ... 2 more fields]
cols: Array[String] = Array(sleeve, normalized_sleeve_0, normalized_sleeve_1, similargrouplevel)
attrDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [sleeve: string, normalized_sleeve_0: double ... 2 more fields]


In [40]:
productsToPredict.join(attrDF, Seq(SIMILAR_GROUP_LEVEL, attribute), "leftanti").show(false)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+----------------+------------------+-----------+---------+-----------+---------+
|similargrouplevel|sleeve          |productid         |colorfamily|brandname|pattern    |styletype|
+-----------------+----------------+------------------+-----------+---------+-----------+---------+
|830303011        |stylised sleeve |460208940_offwhite|white      |ajio     |solid      |straight |
|830303011        |stylised sleeve |440780256_red     |red        |avaasa   |solid      |a-line   |
|830303011        |stylised sleeve |460528805_yellow  |yellow     |indya    |indian     |a-line   |
|830303011        |racerback sleeve|460863887_beige   |beige      |svrnaa   |floral     |a-line   |
|830303011        |stylised sleeve |440932774_navy    |blue       |avaasa   |floral     |flared   |
|830303011        |stylised sleeve |440773531_blue    |blue       |w        |solid      |straight |
|830303011        |stylised sleeve |440875319_pink    |pink       |w        |embellished|straight |


In [41]:
var df = ParquetToDF.getDF(s"$runDir/OriginalDataset/test/XData")
df.filter(col("productid").isin("460208940_offwhite", "440780256_red")).show(false)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

df: org.apache.spark.sql.DataFrame = [productid: string, 0_sales: double ... 155 more fields]
+------------------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+--------+--------+------------------+------------------+-------------------+-------------------+----------------+-----------------+----------------+----------------------+----------------+-----------------+----------------+----------------------+----------------+-----------------+----------------+----------------------+----------------+-----------------+----------------+----------------------+----------------+-----------------+----------------+----------------------+----------------+-----------------+----------------+----------------------+----------------+-----------------+----------------+----------------------+----------------+-----------------+----------------+----------------------+----------------+-----------------+----------------+----------------------+----------------+-----------------+

### Intergration Simulator

In [5]:
var suffix = "2023-04-30/queryProductMap_colorfamily_pricebucket_styletype_pattern_sleeve_brandname"
var suffix2 = "colorfamily_pricebucket_styletype_pattern_sleeve_brandname"

var productAttr = spark.read.parquet("/data/Archive/inventory/"+suffix+"/processedProductAttr").select(col("productid") as "productid",col("similargrouplevel"),col("colorfamily"),col("brandname"),col("styletype"),col("pattern"),col("sleeve").as("sleeve"),col("WeightedAvgPrice").as("price"),col("title"),col("imgcode"))

productAttr = productAttr.withColumn("category_name",when(col("similargrouplevel")==="830216013",lit("Men-Shirts")).when(col("similargrouplevel")==="830303011",lit("Women-Kurtas")))

var queriesDF = spark.read.parquet(s"/data/Archive/bhavesh/InventoryPrediction/2023-04-30/QueryProductMapping/suffix=$suffix2")

var newQueryMap = spark.read.parquet("/data/Archive/inventory/2023-04-30/queryProductMap_colorfamily_pricebucket_styletype_pattern_sleeve_brandname/attributesQueryInfo/final").select("query","template","similargrouplevel","newQuery")

queriesDF = queriesDF.join(newQueryMap,Seq("query","template","similargrouplevel")).withColumnRenamed("query","oldQuery").withColumnRenamed("newQuery","query")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

suffix: String = 2023-04-30/queryProductMap_colorfamily_pricebucket_styletype_pattern_sleeve_brandname
suffix2: String = colorfamily_pricebucket_styletype_pattern_sleeve_brandname
productAttr: org.apache.spark.sql.DataFrame = [productid: string, similargrouplevel: string ... 8 more fields]
productAttr: org.apache.spark.sql.DataFrame = [productid: string, similargrouplevel: string ... 9 more fields]
queriesDF: org.apache.spark.sql.DataFrame = [query: string, productid: string ... 2 more fields]
newQueryMap: org.apache.spark.sql.DataFrame = [query: string, template: string ... 2 more fields]
queriesDF: org.apache.spark.sql.DataFrame = [oldQuery: string, template: string ... 3 more fields]


In [6]:
var predictedqueries = spark.read.parquet("/data/ecomm/ajio/processed/inventory/integrationSimulator/predictions").select("similargrouplevel","query").distinct

var requiredProducts = queriesDF.join(predictedqueries,Seq("similargrouplevel","query")).select("productid").distinct

productAttr.join(requiredProducts,Seq("productid")).write.mode("overwrite").parquet("/data/ecomm/ajio/processed/inventory/integrationSimulator/productDetails")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

predictedqueries: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [similargrouplevel: string, query: string]
requiredProducts: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [productid: string]


In [20]:
requiredProducts.count

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res35: Long = 434089


In [17]:
var queryMonthly =  (
    spark.read.parquet("/data/Archive/bhavesh/InventoryPrediction/2023-04-30/MonthlyQuantity/query")
    .withColumnRenamed("monthsInHis","pastmonth").filter(col("pastmonth")>0)
    .join(newQueryMap,Seq("query","template","similargrouplevel"))
    .withColumnRenamed("query","oldQuery")
    .withColumnRenamed("newQuery","query")
)
(
    queryMonthly
    .withColumn("category_name",when(col("similargrouplevel")==="830216013",lit("Men-Shirts"))
                .when(col("similargrouplevel")==="830303011",lit("Women-Kurtas")))
    .join(predictedqueries,Seq("query","similargrouplevel"))
    .write.mode("overwrite")
    .parquet("/data/ecomm/ajio/processed/inventory/integrationSimulator/monthlySales")  
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

queryMonthly: org.apache.spark.sql.DataFrame = [oldQuery: string, template: string ... 5 more fields]


In [13]:
var df = ParquetToDF.getDF("/data/ecomm/ajio/metadata/styleVectorsViT_large")
df.printSchema

var prodDetails = ParquetToDF.getDF("/data/ecomm/ajio/processed/inventory/integrationSimulator/productDetails").select(col(PRODUCTID).as("itemid"), col("similargrouplevel"))
prodDetails.printSchema

df = df.join(prodDetails, Seq("itemid"))
df = df.withColumn("livestatus", lit(1))
df.printSchema
DFToParquet.putDF("/data/ecomm/ajio/processed/inventory/integrationSimulator/productImageEmbeddings", df)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

df: org.apache.spark.sql.DataFrame = [itemid: string, inferred_vector: array<double>]
root
 |-- itemid: string (nullable = true)
 |-- inferred_vector: array (nullable = true)
 |    |-- element: double (containsNull = true)

prodDetails: org.apache.spark.sql.DataFrame = [itemid: string, similargrouplevel: string]
root
 |-- itemid: string (nullable = true)
 |-- similargrouplevel: string (nullable = true)

df: org.apache.spark.sql.DataFrame = [itemid: string, inferred_vector: array<double> ... 1 more field]
df: org.apache.spark.sql.DataFrame = [itemid: string, inferred_vector: array<double> ... 2 more fields]
root
 |-- itemid: string (nullable = true)
 |-- inferred_vector: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- similargrouplevel: string (nullable = true)
 |-- livestatus: integer (nullable = false)



In [14]:
var productDetails = ParquetToDF.getDF("/data/ecomm/ajio/processed/inventory/integrationSimulator/productDetails")
productDetails.printSchema
println(productDetails.count)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

productDetails: org.apache.spark.sql.DataFrame = [productid: string, similargrouplevel: string ... 9 more fields]
root
 |-- productid: string (nullable = true)
 |-- similargrouplevel: string (nullable = true)
 |-- colorfamily: string (nullable = true)
 |-- brandname: string (nullable = true)
 |-- styletype: string (nullable = true)
 |-- pattern: string (nullable = true)
 |-- sleeve: string (nullable = true)
 |-- price: double (nullable = true)
 |-- title: string (nullable = true)
 |-- imgcode: string (nullable = true)
 |-- category_name: string (nullable = true)

434089


In [18]:
var queryMonthly = ParquetToDF.getDF("/data/ecomm/ajio/processed/inventory/integrationSimulator/monthlySales")
queryMonthly.printSchema
println(queryMonthly.count)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

queryMonthly: org.apache.spark.sql.DataFrame = [query: string, similargrouplevel: int ... 6 more fields]
root
 |-- query: string (nullable = true)
 |-- similargrouplevel: integer (nullable = true)
 |-- oldQuery: string (nullable = true)
 |-- template: string (nullable = true)
 |-- pastmonth: integer (nullable = true)
 |-- quantity: double (nullable = true)
 |-- tempDateForTimeSeries: date (nullable = true)
 |-- category_name: string (nullable = true)

5497095


In [21]:
queryMonthly.show(5, false)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------------------------------------+-----------------+-------------------------------------------------+---------------------------------------------------+---------+--------+---------------------+-------------+
|query                                              |similargrouplevel|oldQuery                                         |template                                           |pastmonth|quantity|tempDateForTimeSeries|category_name|
+---------------------------------------------------+-----------------+-------------------------------------------------+---------------------------------------------------+---------+--------+---------------------+-------------+
|cool-colors_green_check_2_#_classic                |830216013        |check_2_green_classic_cool-colors                |pattern_pricebucket_colorfamily_styletype_brandname|2        |2.0     |2023-03-01           |Men-Shirts   |
|soch_#_indian_3_#_#                                |830303011        |indian_3_soch

In [16]:
var predictions = ParquetToDF.getDF("/data/ecomm/ajio/processed/inventory/integrationSimulator/predictions")
predictions.printSchema
println(predictions.count)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

predictions: org.apache.spark.sql.DataFrame = [similargrouplevel: string, category_name: string ... 7 more fields]
root
 |-- similargrouplevel: string (nullable = true)
 |-- category_name: string (nullable = true)
 |-- query: string (nullable = true)
 |-- m1: double (nullable = true)
 |-- m2: double (nullable = true)
 |-- m3: double (nullable = true)
 |-- m4: double (nullable = true)
 |-- m5: double (nullable = true)
 |-- m6: double (nullable = true)

765221
